# Start-up simulation of a synchronous machine (interactive plots)

Transient simulation is based on stator-oriented $\alpha$-$\beta$ model:
$$
\begin{alignedat}{4}
            &&\text{Stator voltage:}&& \quad\bm{u}^\mathrm{s}_\mathrm{s,\alpha\beta}(&t) &&= R_\mathrm{s} \bm{i}^\mathrm{s}_\mathrm{s,\alpha\beta}(t)+ \frac{\mathrm{d}}{\mathrm{d}t}\bm{\psi}^\mathrm{s}_\mathrm{s,\alpha\beta}(t),\\
            &&\text{Rotor voltage:}&& \quad\bm{u}^\mathrm{s}_\mathrm{r,\alpha\beta}(&t) &&= R_\mathrm{r} \bm{i}^\mathrm{s}_\mathrm{r,\alpha\beta}(t)-\omega_\mathrm{r,el}(t)\bm{J}\bm{\psi}^\mathrm{s}_\mathrm{r,\alpha\beta}(t) +\frac{\mathrm{d}}{\mathrm{d}t}\bm{\psi}^\mathrm{s}_\mathrm{r,\alpha\beta}(t),\\
            &&\text{Stator flux linkage:}&& \quad \bm{\psi}^\mathrm{s}_\mathrm{s,\alpha\beta}(&t) &&= (L_\mathrm{s} +\frac{M_\mathrm{s}}{2}) \bm{i}^\mathrm{s}_{\mathrm{s},\alpha\beta}(t) +  M_{\mathrm{r}}\frac{3}{2}\frac{N_\mathrm{s}}{N_\mathrm{r}} \bm{i}^\mathrm{s}_{\mathrm{r},\alpha\beta}(t),\\
            &&\text{Rotor flux linkage:}&& \quad \bm{\psi}^\mathrm{s}_\mathrm{r,\alpha\beta}(&t) &&= (L_\mathrm{r} +\frac{M_\mathrm{r}}{2}) \bm{i}^\mathrm{s}_{\mathrm{r},\alpha\beta}(t) +  M_{\mathrm{s}}\frac{3}{2}\frac{N_\mathrm{r}}{N_\mathrm{s}} \bm{i}^\mathrm{s}_{\mathrm{s},\alpha\beta}(t),\\
            &&\text{Torque:}&& \quad T(&t) &&= \frac{3}{2} p (\bm{i}_\mathrm{s,\alpha\beta}^\mathrm{s})^\mathsf{T}\bm{J}\bm{\psi}_\mathrm{s,\alpha\beta}^\mathrm{s}=-\frac{3}{2} p (\bm{i}^\mathrm{s}_\mathrm{r,\alpha\beta})^\mathsf{T}\bm{J}\bm{\psi}^\mathrm{s}_\mathrm{r,\alpha\beta}. 
        \end{alignedat}
$$

Three-phase stator voltage supply is given with a fixed amplitude and frequency

$$
\begin{align}
u^\mathrm{s}_{\mathrm{s,a}}(t) &= \hat{u}_\mathrm{s}\sin(\omega t),\\
u^\mathrm{s}_{\mathrm{s,b}}(t) &= \hat{u}_\mathrm{s}\sin(\omega t - \frac{2\pi}{3}),\\
u^\mathrm{s}_{\mathrm{s,c}}(t) &= \hat{u}_\mathrm{s}\sin(\omega t + \frac{2\pi}{3})
\end{align}
$$

while the rotor voltage is zero due to the squirell-cage design: $u^\mathrm{s}_{\mathrm{r,a}}(t)=u^\mathrm{s}_{\mathrm{r,b}}(t)=u^\mathrm{s}_{\mathrm{r,c}}(t)=0$.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import HBox, Layout, VBox, interact, interactive
from scipy.integrate import solve_ivp

In [ ]:
# Simulation settings
t_end = 0.5  # seconds
t_eval = np.linspace(0, t_end, 5000)

# Helper matrix: rotation by 90 degrees
J_mat = np.array([[0, -1], [1, 0]])

In [ ]:
def Clarke_transform(x_a, x_b, x_c):
    x_alpha = (2 / 3) * (x_a - 0.5 * x_b - 0.5 * x_c)
    x_beta = (2 / 3) * (np.sqrt(3) / 2 * x_b - np.sqrt(3) / 2 * x_c)
    return np.array([x_alpha, x_beta])

In [ ]:
def Inverse_Clarke_transform(x_alpha, x_beta):
    x_a = x_alpha
    x_b = -0.5 * x_alpha + np.sqrt(3) / 2 * x_beta
    x_c = -0.5 * x_alpha - np.sqrt(3) / 2 * x_beta
    return np.array([x_a, x_b, x_c])

In [ ]:
def Park_transform(x_alpha, x_beta, theta):
    x_d = x_alpha * np.cos(theta) + x_beta * np.sin(theta)
    x_q = -x_alpha * np.sin(theta) + x_beta * np.cos(theta)
    return np.array([x_d, x_q])

In [ ]:
def Inverse_Park_transform(x_d, x_q, theta):
    x_alpha = x_d * np.cos(theta) - x_q * np.sin(theta)
    x_beta = x_d * np.sin(theta) + x_q * np.cos(theta)
    return np.array([x_alpha, x_beta])

In [ ]:
def stator_voltage(
    t, f=50, V_ll=400
):  # Stator αβ voltages from balanced 3-phase supply at frequency f and line-to-line voltage V_ll
    w_sup = 2 * np.pi * f  # Angular frequency of supply
    V_ph = V_ll / np.sqrt(3)  # Phase voltage

    # Phase voltages in time domain
    ua = V_ph * np.sqrt(2) * np.cos(w_sup * t)
    ub = V_ph * np.sqrt(2) * np.cos(w_sup * t - 2 * np.pi / 3)
    uc = V_ph * np.sqrt(2) * np.cos(w_sup * t + 2 * np.pi / 3)

    # Clark transformation to dq
    return Park_transform(Clarke_transform(ua, ub, uc))

In [ ]:
def flux_to_currents(
    psi_s_s, psi_s_r, Ms, Mr, Ls_sigma, Lr_sigma, Ns, Nr
):  # Calculate motor currents given flux linkages based on flux model
    Ls = Ls_sigma + Ms
    Lr = Lr_sigma + Mr
    A = np.array([[Ls + Ms / 2, 0], [0, Ls + Ms / 2]])
    B = (Mr * 3 / 2 * Ns / Nr) * np.eye(2)
    C = (Ms * 3 / 2 * Nr / Ns) * np.eye(2)
    D = np.array([[Lr + Mr / 2, 0], [0, Lr + Mr / 2]])
    Mmat = np.block([[A, B], [C, D]])
    flux = np.concatenate([psi_s_s, psi_s_r])
    currents = np.linalg.solve(Mmat, flux)
    i_s_s = currents[0:2]
    i_s_r = currents[2:4]
    return i_s_s, i_s_r

In [ ]:
def motor_ode(
    t, x, Rs, Rr, Mr, Ms, Lr_sigma, Ls_sigma, p, J, br, f, V_ll
):  # Flux linkage ODE function utilizing scipy's ODE solver
    # Unpack state
    psi_s_s = x[0:2]
    psi_s_r = x[2:4]
    omega_r = x[4]

    # Self-inductances
    Ls = Ls_sigma + Ms
    Lr = Lr_sigma + Mr

    i_s_s, i_s_r = flux_to_currents(
        psi_s_s, psi_s_r, Ms, Mr, Ls_sigma, Lr_sigma, Ns, Nr
    )  # Calculate stator and rotor currents

    # Voltages
    u_s = stator_voltage(t, f, V_ll)  # Stator voltages in αβ frame
    u_r = np.array([0.0, 0.0])  # Short-circuited rotor (assuming squirrel cage design)

    # Derivatives of fluxes
    dpsi_s_s_dt = u_s - Rs * i_s_s
    dpsi_s_r_dt = u_r - Rr * i_s_r + omega_r * p * J_mat @ psi_s_r

    # Torque
    torque = (3 / 2) * p * i_s_s @ (J_mat @ psi_s_s)

    # Mechanical dynamics (assuming simple friction model depending on rotor speed)
    domega_r_dt = (torque - br * omega_r) / J

    return np.concatenate([dpsi_s_s_dt, dpsi_s_r_dt, [domega_r_dt]])

In [ ]:
def interactive_plot(Rs, Rr, Ms, Mr, Lr_sigma, Ls_sigma, p, J, br, f, V_ll):
    x0 = np.zeros(5)  # Initial flux linkages and rotor speed (all zero)

    # Solve ODE, i.e., simulate the motor dynamics
    sol = solve_ivp(
        motor_ode,
        [0, t_end],
        x0,
        t_eval=t_eval,
        method="RK45",
        args=(Rs, Rr, Mr, Ms, Lr_sigma, Ls_sigma, p, J, br, f, V_ll),
    )

    # Extract solution
    psi_s_s = sol.y[0:2]
    psi_s_r = sol.y[2:4]
    omega_r = sol.y[4]

    i_s_s, i_s_r = flux_to_currents(
        psi_s_s, psi_s_r, Ms, Mr, Ls_sigma, Lr_sigma, Ns, Nr
    )  # Calculate stator and rotor currents

    # Calculate torque from stator currents and stator flux linkages for the entire time series
    torque = (3 / 2) * p * np.sum(i_s_s * (J_mat @ psi_s_s), axis=0)

    # Calculate three-phase currents
    i_s_s_abc = Inverse_Clarke_transform(i_s_s[0], i_s_s[1])
    i_s_r_abc = Inverse_Clarke_transform(i_s_r[0], i_s_r[1])

    # Plotting style
    plt.rc("text", usetex=True)
    font = {"family": "normal", "weight": "normal", "size": 18}

    plt.rc("font", **font)
    plt.figure(figsize=(17, 8))

    # Plot contents
    plt.subplot(4, 2, 1)
    plt.plot(sol.t, psi_s_s[0], label=r"$\psi^\mathrm{s}_{\mathrm{s},\alpha}$")
    plt.plot(sol.t, psi_s_s[1], label=r"$\psi^\mathrm{s}_{\mathrm{s},\beta}$")
    plt.ylabel(r"$\psi^\mathrm{s}_\mathrm{s}(t)$ in Vs")
    plt.legend()
    plt.grid()

    plt.subplot(4, 2, 2)
    plt.plot(sol.t, psi_s_r[0], label=r"$\psi^\mathrm{s}_{\mathrm{r},\alpha}$")
    plt.plot(sol.t, psi_s_r[1], label=r"$\psi^\mathrm{s}_{\mathrm{r},\beta}$")
    plt.ylabel(r"$\psi^\mathrm{s}_\mathrm{r}(t)$ in Vs")
    plt.legend()
    plt.grid()

    plt.subplot(4, 2, 3)
    plt.plot(sol.t, i_s_s[0], label=r"$i^\mathrm{s}_{\mathrm{s},\alpha}$")
    plt.plot(sol.t, i_s_s[1], label=r"$i_{\mathrm{s},\beta}$")
    plt.ylabel(r"$i^\mathrm{s}_\mathrm{s}(t)$ in A")
    plt.legend()
    plt.grid()

    plt.subplot(4, 2, 4)
    plt.plot(sol.t, i_s_r[0], label=r"$i^\mathrm{s}_{\mathrm{r},\alpha}$")
    plt.plot(sol.t, i_s_r[1], label=r"$i^\mathrm{s}_{\mathrm{r},\beta}$")
    plt.ylabel(r"$i^\mathrm{s}_\mathrm{r}(t)$ in A")
    plt.legend()
    plt.grid()

    plt.subplot(4, 2, 5)
    plt.plot(sol.t, i_s_s_abc[0], label=r"$i^\mathrm{s}_{\mathrm{s,a}}$")
    plt.plot(sol.t, i_s_s_abc[1], label=r"$i^\mathrm{s}_{\mathrm{s,b}}$")
    plt.plot(sol.t, i_s_s_abc[2], label=r"$i^\mathrm{s}_{\mathrm{s,c}}$")
    plt.ylabel(r"$i^\mathrm{s}_\mathrm{s}(t)$ in A")
    plt.legend()
    plt.grid()

    plt.subplot(4, 2, 6)
    plt.plot(sol.t, i_s_r_abc[0], label=r"$i^\mathrm{s}_{\mathrm{r,a}}$")
    plt.plot(sol.t, i_s_r_abc[1], label=r"$i^\mathrm{s}_{\mathrm{r,b}}$")
    plt.plot(sol.t, i_s_r_abc[2], label=r"$i^\mathrm{s}_{\mathrm{r,c}}$")
    plt.ylabel(r"$i^\mathrm{s}_\mathrm{r}(t)$ in A")
    plt.legend()
    plt.grid()

    plt.subplot(4, 2, 7)
    plt.plot(sol.t, omega_r * 60 / (2 * np.pi), label="Rotor Speed")
    plt.ylabel(r"$n_\mathrm{me}(t)$ in min$^{-1}$")
    plt.xlabel("t in s")
    plt.grid()

    plt.subplot(4, 2, 8)
    plt.plot(sol.t, torque, label="Electromagnetic Torque")
    plt.ylabel("$T(t)$ in Nm")
    plt.xlabel("t in s")
    plt.grid()

    plt.tight_layout()
    plt.show()

In [ ]:
# Create interactive widget for the motor simulation with changable parameters and supply voltage
widget = interactive(
    interactive_plot,
    Rs=(2.0, 3.0, 0.1),  # Stator resistance
    Rr=(2.0, 3.0, 0.1),  # Rotor resistance
    Mr=(0.1, 0.5, 0.01),  # Rotor mutual inductance
    Ms=(0.1, 0.5, 0.01),  # Stator mutual inductance
    Lr_sigma=(0.01, 0.05, 0.01),  # Rotor leakage inductance
    Ls_sigma=(0.01, 0.05, 0.01),  # Stator leakage inductance
    p=(1, 3, 1),  # Number of pole pairs
    J=(0.01, 0.1, 0.005),  # Rotor inertia
    br=(00, 0.4, 0.01),  # Rotor friction coefficient
    f=(0, 100, 1),  # Stator supply frequency
    V_ll=(0, 800, 10),
)  # Stator line-to-line voltage

controls = HBox(widget.children[:-1], layout=Layout(flex_flow="row wrap"))
output = widget.children[-1]
display(VBox([controls, output]))